## Vistual inertia scheduling

vittual inertia scheduling (vis) is inherited from dcopf in opf.py

base class: dcopf <br>
vis1: dcopf + Pvsg <br>
vis2: dcopf + RoCof and fnadir + Pvsg <br>


In [702]:
import andes
import os

from statistics import fmean
from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import gurobipy as gb
import pandas as pd
import numpy as np
import logging

logger = logging.getLogger(__name__)

from opf import dcopf

In [703]:
class vis1(dcopf):
    """
    vis0: fixed vsg up and down reserve
    """

    def __init__(self, name='dcopf', norm=None, nn=None, nn_num=64):
        """
        Parameter
        ---------
        name: str
            name 
        norm: dict
            normalization dict for function fnaidir and ppeak 
            {
                Mvsg: [mean, std],
                Dvsg: [mean, std],
                Fg: [mean, std],
                Rg: [mean, std],
                Minv: [mean, std],
                Dinv: [mean, std],
            }
        nn: neural
        nn_num: integer
            number of MLP nuerols (assume single layer MLP)
        """
        super().__init__(name)
        self.norm = norm # TO DO ...
        self.nn = nn # TO DO ...
        self.nn_num = nn_num


    def from_andes(self, ssa, typeII=None):
        """
        ssa: andes model
        typeII: idx of typeII generator, vsg inverter
            i.g. ['PV_6', 'PV_7']
        """      
        super().from_andes(ssa)

        # define typeII, defalt typeI: type=1
        self.gen['type'] = 1
        if typeII:
            for idx in typeII:
                row = self.gen[self.gen['idx'] == idx].index[0]
                self.gen['type'].iloc[row] = 2

        self.gen['p_pre'] = 0
        self.gen['band'] = self.gen['pmax'] - self.gen['pmin']
        self.gen['Sn'] /= self.mva  # normalize Sn
        self.gen['fg'] = 1
        # self.gen['M'] = 0
        # self.gen['D'] = 0
        # self.gen['R'] = 0.05
        # self.gen['Mvsg'] = 0
        # self.gen['Dvsg'] = 0

        genrow = ssa.GENROU.as_df()
        regc = ssa.REGCV2.as_df()
        tgov = ssa.TGOV1N.as_df()
        tgov.rename(columns={'idx':'gov', 'syn':'idx'}, inplace=True)
        regc.rename(columns={'idx':'vsg', 'gen':'idx', 'M':'Mvsg', 'D': 'Dvsg'},  inplace=True)

        # merge tgov R to genrow based on idex in tgover and syn in tgov
        genrow = pd.merge(left=genrow, right=tgov[['idx', 'R']], on='idx', how='left')

        # merge M, D, R in genrow to self.gen based on gen
        genrow.rename(columns={'idx': 'syn', 'gen': 'idx'}, inplace=True)
        self.gen = pd.merge(left=self.gen, right=genrow[['idx', 'M','D', 'R']], on='idx', how='left')

        # merge Mvsg, Dvsg, in genrow to self.gen based on gen
        self.gen = pd.merge(left=self.gen, right=regc[['idx', 'Mvsg', 'Dvsg']], on='idx', how='left')

        self.gen.fillna(0, inplace=True)

        # update dict after revising pandas dataframe
        self.update_dict()


    def build(self):
        # self.data_check()

        # --- build gurobi model ---
        self.update_dict()
        self.mdl = gb.Model(self.name)
        self.mdl = self._build_vars(self.mdl)
        self.mdl = self._build_obj(self.mdl)
        self.mdl = self._build_cons(self.mdl)
        logger.info('Successfully build vis0 model.')


    def _build_vars(self, mdl):
        GEN = self.gendict.keys()

        # --- uncontrollable generators limit to p0 ---
        gencp = self.gen.copy()
        gencp['pmax'][gencp.ctrl == 0] = gencp['p0'][gencp.ctrl == 0]
        gencp['pmin'][gencp.ctrl == 0] = gencp['p0'][gencp.ctrl == 0]
        # --- offline geenrators limit to 0 ---
        gencp['pmax'][gencp.u == 0] = 0
        gencp['pmin'][gencp.u == 0] = 0

        # --- gen: pg ---
        self.pg = mdl.addVars(GEN, name='pg', vtype=gb.GRB.CONTINUOUS, obj=0,
                              ub=gencp.pmax.tolist(), lb=gencp.pmin.tolist())
        # --- RegUp, RegDn --- !!!modify inverter reserve up and down in andes file, prd=0
        self.pru = mdl.addVars(GEN, name='pru', vtype=gb.GRB.CONTINUOUS, obj=0,
                               ub=gencp.band.tolist(), lb=[0] * gencp.shape[0])
        self.prd = mdl.addVars(GEN, name='prd', vtype=gb.GRB.CONTINUOUS, obj=0,
                               ub=gencp.band.tolist(), lb=[0] * gencp.shape[0])
        
        # --- Mvsg, Dvsg ---
        gendict = self.gendict
        gendict_II = dict()
        for (new_key, new_value) in gendict.items():
            if new_value['type'] == 2:
                gendict_II[new_key] = new_value
        vsg = gendict_II.keys()

        self.Mvsg = mdl.addVars(vsg, name='Mvsg', vtype=gb.GRB.CONTINUOUS, obj=0,
                               ub=[4]*len(vsg), lb=[0]*len(vsg))
        self.Dvsg = mdl.addVars(vsg, name='Dvsg', vtype=gb.GRB.CONTINUOUS, obj=0,
                               ub=[5]*len(vsg), lb=[0]*len(vsg))

        # --- a and z for ml assisted linearization ---
        # 'ap, zp' for vsg power; 'af, zf' for frequency nadir
        ap = []
        zp = []
        for i in range(self.nn_num):
            ap.append('ap'+str(i))
            zp.append('zp'+str(i))
        self.ap = mdl.addVars(ap, name='ap', vtype=gb.GRB.BINARY)
        self.zp = mdl.addVars(zp, name='zp', vtype=gb.GRB.CONTINUOUS)

        return mdl

    def _build_cons(self, mdl):
        # --- filter Type II gen ---
        gendict = self.gendict
        gendict_I = dict()
        for (new_key, new_value) in gendict.items():
            if new_value['type'] == 1:
                gendict_I[new_key] = new_value
        gendict_II = dict()
        for (new_key, new_value) in gendict.items():
            if new_value['type'] == 2:
                gendict_II[new_key] = new_value
        GENI = gendict_I.keys()
        GENII = gendict_II.keys()

        # Synthetic M/D/F/R
        MI = sum(gendict[gen]['Sn'] * gendict[gen]['M'] for gen in GENI)
        MI += sum(gendict[gen]['Sn'] * gendict[gen]['Mvsg'] for gen in GENII)

        


## Main (Test)

In [704]:
# get andes case from excel
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee14_vis.xlsx'
case = dir_path + case_path
ssa = andes.load(case, no_output=True)

REGCV2: unused data {'KpId': 50, 'KiId': 100, 'KpIq': 50, 'KiIq': 100}
REGCV2: unused data {'KpId': 50, 'KiId': 100, 'KpIq': 50, 'KiIq': 100}


In [705]:
ss = vis1()

In [706]:
ss.from_andes(ssa, ['PV_6', 'PV_7'])

In [707]:
ss.gen

,idx,u,name,Sn,Vn,bus,p0,pmax,pmin,v0,...,ramp30,type,p_pre,band,fg,M,D,R,Mvsg,Dvsg
0,PV_2,1.0,PV_2,1.0,69.0,2,0.40000,0.5,0.1,1.03,...,600,1,0,0.4,1,8.0,0.0,0.12,0.0,0.0
1,PV_3,1.0,PV_3,1.0,69.0,3,0.40000,0.5,0.1,1.01,...,600,1,0,0.4,1,5.0,0.0,0.12,0.0,0.0
2,PV_4,1.0,PV_4,1.0,138.0,6,0.30000,1.0,0.1,1.03,...,600,1,0,0.9,1,5.0,0.0,0.05,0.0,0.0
3,PV_5,1.0,PV_5,1.0,69.0,8,0.30000,0.5,0.1,1.03,...,600,1,0,0.4,1,10.0,0.0,0.05,0.0,0.0
4,PV_6,1.0,PV_6,1.0,138.0,14,0.10000,0.1,-0.1,1.01,...,600,2,0,0.2,1,0.0,0.0,0.00,0.0,3.0
5,PV_7,1.0,PV_7,1.0,138.0,12,0.10000,0.1,-0.1,1.01,...,600,2,0,0.2,1,0.0,0.0,0.00,6.0,2.0
6,Slack_1,1.0,Slack_1,1.0,69.0,1,0.81442,3.0,0.5,1.03,...,600,1,0,2.5,1,8.0,0.0,0.12,0.0,0.0


In [708]:
# ss.gendict

In [709]:
ss.norm

In [710]:
a_test = []
for i in range(64):
    a_test.append('a'+str(i))
# a_test

In [711]:
ss._build_vars(ss.mdl)

<gurobi.Model Continuous instance dcopf: 0 constrs, 0 vars, No parameter changes>

In [712]:
ss.pg

{'PV_2': <gurobi.Var *Awaiting Model Update*>,
 'PV_3': <gurobi.Var *Awaiting Model Update*>,
 'PV_4': <gurobi.Var *Awaiting Model Update*>,
 'PV_5': <gurobi.Var *Awaiting Model Update*>,
 'PV_6': <gurobi.Var *Awaiting Model Update*>,
 'PV_7': <gurobi.Var *Awaiting Model Update*>,
 'Slack_1': <gurobi.Var *Awaiting Model Update*>}

In [713]:
# ss.Mvsg

In [714]:
# ss.pru

In [715]:
# display gurobi formulation
ss.mdl.display()

Minimize
  <gurobi.LinExpr: 0.0>
Subject To
